In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279205,85.324353,83.909682,84.556820,118944600
1,2010-01-05,85.504936,85.542563,84.917998,85.226520,111579900
2,2010-01-06,85.565147,85.775842,85.354452,85.422173,116074400
3,2010-01-07,85.926346,86.031693,85.166334,85.407129,131091100
4,2010-01-08,86.212257,86.249884,85.527499,85.700567,126402800


In [3]:

# A common pattern for Pine Script to Python is to use a class to encapsulate the indicator's state
# and logic, processing a DataFrame of financial data bar-by-bar.

class RangeDetector:
    def __init__(self, rd_length=20, rd_mult=1.0, rd_atrLen=500):
        """
        Initializes the Range Detector with user-defined parameters.

        Args:
            rd_length (int): Minimum range length.
            rd_mult (float): Range width multiplier for ATR.
            rd_atrLen (int): Length for the Average True Range (ATR) calculation.
        """
        self.rd_length = rd_length
        self.rd_mult = rd_mult
        self.rd_atrLen = rd_atrLen
        
        # State variables from Pine Script's 'var' keyword.
        # In Python, these are instance attributes that persist across bar-by-bar calculations.
        self.bx = {'left': np.nan, 'top': np.nan, 'right': np.nan, 'bottom': np.nan}
        self.lvl = {'x1': np.nan, 'y1': np.nan, 'x2': np.nan, 'y2': np.nan}
        self.rd_max = np.nan
        self.rd_min = np.nan
        self.rd_os = 0
        self.detect_css = None
        self.unbroken_css = '#2157f3' # This is just a string, as drawing isn't native to Python script
        self.rd_up_css = '#089981'
        self.rd_dn_css = '#f23645'
        
        # Initialize ATR and SMA calculation results
        self.atr_series = pd.Series(dtype=float)
        self.ma_series = pd.Series(dtype=float)

    def calculate(self, data, switch_rd=True):
        """
        Calculates the Range Detector indicator for a DataFrame.

        Args:
            data (pd.DataFrame): DataFrame containing 'close', 'high', and 'low' prices.
            switch_rd (bool): Determines if range detection is active.
        
        Returns:
            A tuple of pandas Series: (rd_max, rd_min, rd_os, detect_css).
        """
        self.atr_series = self._calculate_atr(data, self.rd_atrLen) * self.rd_mult
        self.ma_series = self._calculate_sma(data, self.rd_length)
        
        rd_max_values = []
        rd_min_values = []
        rd_os_values = []
        detect_css_values = []
        
        for i in range(len(data)):
            # bar_index in Pine corresponds to the index of the DataFrame
            bar_index = i
            close = data['Close'].iloc[i]
            
            # Use data history for calculations
            if i >= self.rd_atrLen:
                self._process_bar(data, i, switch_rd)
            
            # Append current state values
            rd_max_values.append(self.rd_max)
            rd_min_values.append(self.rd_min)
            rd_os_values.append(self.rd_os)
            detect_css_values.append(self.detect_css)
            
        return (
            pd.Series(rd_max_values, index=data.index),
            pd.Series(rd_min_values, index=data.index),
            pd.Series(rd_os_values, index=data.index),
            pd.Series(detect_css_values, index=data.index)
        )

    def _process_bar(self, data, i, switch_rd):
        """Processes a single bar of data to update the indicator's state."""
        close = data['Close'].iloc[i]
        
        # Calculate `count` from Pine Script logic
        sub_series = data['Close'].iloc[i - self.rd_length + 1 : i + 1]
        ma_sub_series = self.ma_series.iloc[i]
        atr_sub_series = self.atr_series.iloc[i]
        count = sum(np.abs(sub_series - ma_sub_series) > atr_sub_series)
        
        # Check `count` from previous bar
        count_prev = sum(np.abs(data['Close'].iloc[i - self.rd_length: i] - self.ma_series.iloc[i - 1]) > self.atr_series.iloc[i - 1]) if i > 0 else -1

        if count == 0 and count_prev != count:
            if i >= self.bx['right'] and not np.isnan(self.bx['right']):
                # Test for overlap and change coordinates
                self.rd_max = max(self.ma_series.iloc[i] + self.atr_series.iloc[i], self.bx['top'])
                self.rd_min = min(self.ma_series.iloc[i] - self.atr_series.iloc[i], self.bx['bottom'])
                
                if switch_rd:
                    # Update existing box
                    self.bx['top'] = self.rd_max
                    self.bx['right'] = i
                    self.bx['bottom'] = self.rd_min
                    # Update level
                    avg = np.mean([self.rd_max, self.rd_min])
                    self.lvl['y1'] = avg
                    self.lvl['x2'] = i
                    self.lvl['y2'] = avg
            else:
                self.rd_max = self.ma_series.iloc[i] + self.atr_series.iloc[i]
                self.rd_min = self.ma_series.iloc[i] - self.atr_series.iloc[i]
                
                if switch_rd:
                    # Create new box and level
                    self.bx = {
                        'left': i - self.rd_length,
                        'top': self.rd_max,
                        'right': i,
                        'bottom': self.rd_min,
                    }
                    self.lvl = {
                        'x1': i - self.rd_length,
                        'y1': self.ma_series.iloc[i],
                        'x2': i,
                        'y2': self.ma_series.iloc[i],
                    }
                    self.detect_css = '#c0c0c0' # Corresponds to a gray background
                    self.rd_os = 0
        
        elif count == 0 and not np.isnan(self.bx['right']):
            # Extend existing box
            self.bx['right'] = i
            self.lvl['x2'] = i

        # Set color based on price relative to the detected range
        if not np.isnan(self.bx['top']):
            if close > self.bx['top']:
                self.rd_os = 1
            elif close < self.bx['bottom']:
                self.rd_os = -1
            else:
                self.rd_os = 0

    def _calculate_atr(self, data, length):
        """Calculates Average True Range using pandas."""
        high = data['High']
        low = data['Low']
        close = data['Close']
        
        tr1 = high - low
        tr2 = np.abs(high - close.shift(1))
        tr3 = np.abs(low - close.shift(1))
        true_range = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
        return true_range.ewm(span=length, adjust=False).mean()

    def _calculate_sma(self, data, length):
        """Calculates Simple Moving Average using pandas."""
        return data['Close'].rolling(window=length).mean()



In [6]:
# Example usage with sample data:
if __name__ == '__main__':
    
    # Instantiate the indicator
    range_detector = RangeDetector()

    # Calculate the indicator values
    rd_max, rd_min, rd_os, detect_css = range_detector.calculate(rawdf, switch_rd=True)

    df['rd_max'] = rd_max
    df['rd_min'] = rd_min
    df['rd_os'] = rd_os
    df['detect_css'] = detect_css


In [7]:
# Range Detector Visualization
if __name__ == '__main__':
    
    chart = Chart(title="Range Detector", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    rdMax_line = chart.create_line('rd_max', color="#ff0000", width=1, price_line=False, price_label=False)
    rdMax_line.set(df[['Date', 'rd_max']])
    rdMin_line = chart.create_line('rd_min', color="#0000ff", width=1, price_line=False, price_label=False)
    rdMin_line.set(df[['Date', 'rd_min']])
    

chart.show(block = True)